# 😃 Evaluate OCR API with Specific Test Dataset

## 🥸 Metrics for Evaluate

In [127]:
import editdistance

ground_truth = "นายนนท์ปวิธ"
predicted_text = "นายนนปวิธ"

### 🥹 Character Error Rate

In [128]:
total_chars = sum(len(gt) for gt in ground_truth)
cer = sum(editdistance.eval(gt, pt) for gt, pt in zip(ground_truth, predicted_text)) / total_chars
print(f"Character Error Rate (CER): {cer * 100:.2f}%")

Character Error Rate (CER): 36.36%


### 😉 Word Error Rate

In [129]:
ground_words = [gt.split() for gt in ground_truth]
predicted_words = [pt.split() for pt in predicted_text]
total_words = sum(len(words) for words in ground_words)
wer = sum(editdistance.eval(gt, pt) for gt, pt in zip(ground_words, predicted_words)) / total_words
print(f"Word Error Rate (WER): {wer * 100:.2f}%")

Word Error Rate (WER): 36.36%


## 😈 OCR API from Google Vision API


In [130]:
# Setting key
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../keys/subtle-bus-443807-m7-63c3367e6ece.json"

In [131]:
from google.cloud import vision
import io

def detect_text(image_path):
    # Create client for Google Vision
    client = vision.ImageAnnotatorClient()
    # Load image
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    # Calling OCR function
    response = client.text_detection(image=image)
    texts = response.text_annotations
    if response.error.message:
        raise Exception(f"API Error: {response.error.message}")
    return texts[0].description 

# Calling detect_text()
predicted_text = detect_text("../../assets/output/angsa_2.png")
print(predicted_text.replace('\n', ' '))

ชื่อ สุไฮลัน สุวรรณหงษ์ บัตรประจำตัว 5518397643662 วันเกิด 27 ธันวาคม 2479 REG020 วันผลิต 2024-07-03 วันหมดอายุ 2027-03-06 CAPSULE PARACETAMOL 5mg ใช้ยาทาเพียงบางๆ นวดเบาๆ วันละ 1-2 ครั้ง ย รักษาภาวะติดเชื้อผิวหนัง ห้ามใช้ในหญิงตั้งครรภ์


In [132]:
with open("../../assets/output/angsa_2.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
print(ground_truth_text.replace('\n', ' '))

ชื่อ สุไฮลัน สุวรรณหงษ์ บัตรประจำตัว 5518397643662 วันเกิด 27 ธันวาคม 2479 REG020 วันผลิต 2024-07-03 วันหมดอายุ 2027-03-06 PARACETAMOL 5mg CAPSULE ใช้ยาทาเพียงบางๆ นวดเบาๆ วันละ 1-2 ครั้ง รักษาภาวะติดเชื้อผิวหนัง ห้ามใช้ในหญิงตั้งครรภ์ 


In [133]:
# Function to calculate Character Error Rate
def calculate_cer(predicted_text, ground_truth_text):
    return editdistance.eval(predicted_text, ground_truth_text) / len(ground_truth_text)

cer = calculate_cer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
print(f"CER: {cer:.2%}")

CER: 8.05%


In [134]:
sum_character_error_rate = 0

for i in range(10):
    predicted_text = detect_text(f"../../assets/output/angsa_{i}.png")
    with open(f"../../assets/output/angsa_{i}.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
    cer = calculate_cer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
    sum_character_error_rate += cer

average_character_error_rate = sum_character_error_rate/10
print(f"Average CER: {cer:.2%}")

Average CER: 10.39%


## 😈 OCR API from PyTesseract


In [135]:
import pytesseract
from PIL import Image

def ocr_image(image_path, lang='tha+eng'):
    # Load Image
    image = Image.open(image_path)
    # Calling OCR function
    text = pytesseract.image_to_string(image, lang=lang)
    return text

In [136]:
print(ocr_image("../../assets/output/angsa_2.png").replace('\n', ' '))

ชื่อ สุไฮลัน สุวรรณหงษ์             REG020 บัตรประจําตัว 5518397643662      วันผลิต 2024-07-03  วันเกิด 27 ธันวาคม 2479           วันหมดอายุ 2027-03-06  PARACETAMOL  5mg         CAPSULE ใช้ยาทาเพียงบางๆ นวดเบาๆ วันละ 1-2 ครั้ง รักษาภาวะติดเชือผิวหนัง  2 ห้ามใช้ในหญิงตั้งครรภ์ 


In [137]:
with open("../../assets/output/angsa_2.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
print(ground_truth_text.replace('\n',' '))

ชื่อ สุไฮลัน สุวรรณหงษ์ บัตรประจำตัว 5518397643662 วันเกิด 27 ธันวาคม 2479 REG020 วันผลิต 2024-07-03 วันหมดอายุ 2027-03-06 PARACETAMOL 5mg CAPSULE ใช้ยาทาเพียงบางๆ นวดเบาๆ วันละ 1-2 ครั้ง รักษาภาวะติดเชื้อผิวหนัง ห้ามใช้ในหญิงตั้งครรภ์ 


In [138]:
sum_character_error_rate = 0

for i in range(10):
    predicted_text = ocr_image(f"../../assets/output/angsa_{i}.png")
    with open(f"../../assets/output/angsa_{i}.txt", encoding="utf-8") as file:
        ground_truth_text = file.read()
    cer = calculate_cer(predicted_text.replace('\n', ' '), ground_truth_text.replace('\n', ' '))
    sum_character_error_rate += cer

average_character_error_rate = sum_character_error_rate/10
print(f"Average CER: {cer:.2%}")

Average CER: 67.53%
